In [67]:
# coding: utf-8

# # Spam classification

# ## Import libraries

# In[5]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import time
import collections
import re
import random
import scipy.io
import glob
from sklearn.model_selection import train_test_split
from pandas_ml import ConfusionMatrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC, SVC
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score
from sklearn.feature_selection import SelectKBest
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from nltk import PorterStemmer

In [6]:
#setting the path for the location
import os
os.chdir('D:/Freelancer_questions/Spam_ham/Enron-Email-Classification-master/Enron-Email-Classification-master')

In [11]:
# ## Vectorizer
#reading the input data from the txt spam vs ham file
# In[8]:

NUM_TRAINING_EXAMPLES = 5172
NUM_TEST_EXAMPLES = 5857

BASE_DIR = './'
SPAM_DIR = 'spam/'
HAM_DIR = 'ham/'
TEST_DIR = 'test/'

vectorizer = TfidfVectorizer(input='filename',lowercase=True, stop_words="english",
                             encoding='latin-1',min_df=8) 

spam_filenames = glob.glob( BASE_DIR + SPAM_DIR + '*.txt')
ham_filenames = glob.glob( BASE_DIR + HAM_DIR + '*.txt')
test_filenames = [BASE_DIR + TEST_DIR + str(x) + '.txt' for x in range(NUM_TEST_EXAMPLES)]
all_filenames = spam_filenames + ham_filenames # including test_filenames

In [13]:
#converted the data to countvectorizer and created a dictionary on the same#
train_matrix = vectorizer.fit_transform(all_filenames)
test_matrix = vectorizer.transform(test_filenames)
X = train_matrix
Y = [1]*len(spam_filenames) + [0]*len(ham_filenames)

# Save as .mat 
file_dict = {}
file_dict['training_data'] = X
file_dict['training_labels'] = Y
file_dict['test_data'] = test_matrix
scipy.io.savemat('email_data.mat', file_dict)

In [21]:
#checking the dictionary sample
file_dict['training_data'].todense()

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.03169233, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.03007344, 0.        , ..., 0.        , 0.        ,
         0.        ]])

In [50]:
train_X = data['training_data'].toarray()
train_y = data['training_labels'].reshape(X.shape[0],1)
X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.20, random_state=42)
test_X = data['test_data'].toarray()
dt = DecisionTreeClassifier() 
    
clf = AdaBoostClassifier(n_estimators=50, base_estimator=dt,learning_rate=1).fit(X_train, y_train)
predicted_test = clf.predict(test_X)
predicted_val = clf.predict(X_val)

C:\Users\ragha\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [68]:
data = scipy.io.loadmat('./email_data.mat')
def adaboost_submission(data):
    # combine test and training data for scaling
    
    train_X = data['training_data'].toarray()
    train_y = data['training_labels'].reshape(X.shape[0],1)
    X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.20, random_state=42)
    test_X = data['test_data'].toarray()
    dt = DecisionTreeClassifier() 
    
    clf = AdaBoostClassifier(n_estimators=50, base_estimator=dt,learning_rate=1).fit(X_train, y_train)
    predicted_test = clf.predict(test_X)
    predicted_val = clf.predict(X_val)
 

    print ('Accuracy:', accuracy_score(y_val, predicted_val))
    print ('F1 score:', f1_score(y_val, predicted_val))
    print ('Recall:', recall_score(y_val, predicted_val))
    print ('Precision:', precision_score(y_val, predicted_val))
    print ('\n clasification report:\n', classification_report(y_val, predicted_val))
    print ('\n confussion matrix:\n',confusion_matrix(y_val, predicted_val))
    
    return predicted_test

submit_ada = adaboost_submission(data)

Accuracy: 0.9642512077294686
F1 score: 0.9417322834645668
Recall: 0.9492063492063492
Precision: 0.934375

 clasification report:
               precision    recall  f1-score   support

           0       0.98      0.97      0.97       720
           1       0.93      0.95      0.94       315

   micro avg       0.96      0.96      0.96      1035
   macro avg       0.96      0.96      0.96      1035
weighted avg       0.96      0.96      0.96      1035


 confussion matrix:
 [[699  21]
 [ 16 299]]


In [69]:
def lda_submission(data):
    # combine test and training data for scaling
    train_X = data['training_data'].toarray()
    train_y = data['training_labels'].reshape(X.shape[0],1)
    test_X = data['test_data'].toarray()
    X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.20, random_state=42)
    clf = LinearDiscriminantAnalysis().fit(X_train, y_train)
    predicted_test = clf.predict(test_X)
    predicted_val = clf.predict(X_val)
 

    print ('Accuracy:', accuracy_score(y_val, predicted_val))
    print ('F1 score:', f1_score(y_val, predicted_val))
    print ('Recall:', recall_score(y_val, predicted_val))
    print ('Precision:', precision_score(y_val, predicted_val))
    print ('\n clasification report:\n', classification_report(y_val, predicted_val))
    print ('\n confussion matrix:\n',confusion_matrix(y_val, predicted_val))
    
    return predicted_test

submit_lda = lda_submission(data)

Accuracy: 0.8830917874396135
F1 score: 0.7979966611018363
Recall: 0.7587301587301587
Precision: 0.8415492957746479

 clasification report:
               precision    recall  f1-score   support

           0       0.90      0.94      0.92       720
           1       0.84      0.76      0.80       315

   micro avg       0.88      0.88      0.88      1035
   macro avg       0.87      0.85      0.86      1035
weighted avg       0.88      0.88      0.88      1035


 confussion matrix:
 [[675  45]
 [ 76 239]]


In [70]:
def qda_submission(data):
    # combine test and training data for scaling
    train_X = data['training_data'].toarray()
    train_y = data['training_labels'].reshape(X.shape[0],1)
    test_X = data['test_data'].toarray()
    X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.20, random_state=42)
    clf = QuadraticDiscriminantAnalysis().fit(X_train, y_train)
    predicted_test = clf.predict(test_X)
    predicted_val = clf.predict(X_val)
 

    print ('Accuracy:', accuracy_score(y_val, predicted_val))
    print ('F1 score:', f1_score(y_val, predicted_val))
    print ('Recall:', recall_score(y_val, predicted_val))
    print ('Precision:', precision_score(y_val, predicted_val))
    print ('\n clasification report:\n', classification_report(y_val, predicted_val))
    print ('\n confussion matrix:\n',confusion_matrix(y_val, predicted_val))
    
    return predicted_test

submit_qda = qda_submission(data)

Accuracy: 0.7265700483091787
F1 score: 0.6802259887005649
Recall: 0.9555555555555556
Precision: 0.5280701754385965

 clasification report:
               precision    recall  f1-score   support

           0       0.97      0.63      0.76       720
           1       0.53      0.96      0.68       315

   micro avg       0.73      0.73      0.73      1035
   macro avg       0.75      0.79      0.72      1035
weighted avg       0.84      0.73      0.74      1035


 confussion matrix:
 [[451 269]
 [ 14 301]]


In [71]:
def xgboost_submission(data):
    # combine test and training data for scaling
    train_X = data['training_data'].toarray()
    train_y = data['training_labels'].reshape(X.shape[0],1)
    test_X = data['test_data'].toarray()
    X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.20, random_state=42)
    clf = XGBClassifier( max_depth=5,min_child_weight=1,gamma=0,subsample=0.8,colsample_bytree=0.8).fit(X_train, y_train)
    predicted_test = clf.predict(test_X)
    predicted_val = clf.predict(X_val)
 

    print ('Accuracy:', accuracy_score(y_val, predicted_val))
    print ('F1 score:', f1_score(y_val, predicted_val))
    print ('Recall:', recall_score(y_val, predicted_val))
    print ('Precision:', precision_score(y_val, predicted_val))
    print ('\n clasification report:\n', classification_report(y_val, predicted_val))
    print ('\n confussion matrix:\n',confusion_matrix(y_val, predicted_val))
    
    return predicted_test

submit_xgm = xgboost_submission(data)

Accuracy: 0.9729468599033816
F1 score: 0.9569230769230769
Recall: 0.9873015873015873
Precision: 0.9283582089552239

 clasification report:
               precision    recall  f1-score   support

           0       0.99      0.97      0.98       720
           1       0.93      0.99      0.96       315

   micro avg       0.97      0.97      0.97      1035
   macro avg       0.96      0.98      0.97      1035
weighted avg       0.97      0.97      0.97      1035


 confussion matrix:
 [[696  24]
 [  4 311]]


In [72]:
def randomforest_submission(data):
    # combine test and training data for scaling
    train_X = data['training_data'].toarray()
    train_y = data['training_labels'].reshape(X.shape[0],1)
    test_X = data['test_data'].toarray()
    X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.20, random_state=42)
    clf = RandomForestClassifier(n_estimators=100).fit(X_train, y_train)
    
    print ('Accuracy:', accuracy_score(y_val, predicted_val))
    print ('F1 score:', f1_score(y_val, predicted_val))
    print ('Recall:', recall_score(y_val, predicted_val))
    print ('Precision:', precision_score(y_val, predicted_val))
    print ('\n clasification report:\n', classification_report(y_val, predicted_val))
    print ('\n confussion matrix:\n',confusion_matrix(y_val, predicted_val))
    
    return predicted_test

submit_rf = randomforest_submission(data)


# ### Logistic Regression

# In[13]:

def logreg_submission(data):
    # combine test and training data for scaling
    train_X = data['training_data'].toarray()
    train_y = data['training_labels'].reshape(X.shape[0],1)
    test_X = data['test_data'].toarray()
    X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.20, random_state=42)
    clf = LogisticRegression(C=1).fit(X_train, y_train)
    
    print ('Accuracy:', accuracy_score(y_val, predicted_val))
    print ('F1 score:', f1_score(y_val, predicted_val))
    print ('Recall:', recall_score(y_val, predicted_val))
    print ('Precision:', precision_score(y_val, predicted_val))
    print ('\n clasification report:\n', classification_report(y_val, predicted_val))
    print ('\n confussion matrix:\n',confusion_matrix(y_val, predicted_val))
    
    return predicted_test

submit_log = logreg_submission(data)

Accuracy: 0.9584541062801932
F1 score: 0.930756843800322
Recall: 0.9174603174603174
Precision: 0.9444444444444444

 clasification report:
               precision    recall  f1-score   support

           0       0.96      0.98      0.97       720
           1       0.94      0.92      0.93       315

   micro avg       0.96      0.96      0.96      1035
   macro avg       0.95      0.95      0.95      1035
weighted avg       0.96      0.96      0.96      1035


 confussion matrix:
 [[703  17]
 [ 26 289]]
Accuracy: 0.9584541062801932
F1 score: 0.930756843800322
Recall: 0.9174603174603174
Precision: 0.9444444444444444

 clasification report:
               precision    recall  f1-score   support

           0       0.96      0.98      0.97       720
           1       0.94      0.92      0.93       315

   micro avg       0.96      0.96      0.96      1035
   macro avg       0.95      0.95      0.95      1035
weighted avg       0.96      0.96      0.96      1035


 confussion matrix:
 [

In [73]:
def svm_submission(data, c):
    # combine test and training data for scaling
    train_X = data['training_data'].toarray()
    train_y = data['training_labels'].reshape(X.shape[0],1)
    test_X = data['test_data'].toarray()
    X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.20, random_state=42)
    clf = LinearSVC(C=c).fit(X_train, y_train)
    
    print ('Accuracy:', accuracy_score(y_val, predicted_val))
    print ('F1 score:', f1_score(y_val, predicted_val))
    print ('Recall:', recall_score(y_val, predicted_val))
    print ('Precision:', precision_score(y_val, predicted_val))
    print ('\n clasification report:\n', classification_report(y_val, predicted_val))
    print ('\n confussion matrix:\n',confusion_matrix(y_val, predicted_val))
    
    return predicted_test
submit_svm = svm_submission(data, 0.1)

Accuracy: 0.9584541062801932
F1 score: 0.930756843800322
Recall: 0.9174603174603174
Precision: 0.9444444444444444

 clasification report:
               precision    recall  f1-score   support

           0       0.96      0.98      0.97       720
           1       0.94      0.92      0.93       315

   micro avg       0.96      0.96      0.96      1035
   macro avg       0.95      0.95      0.95      1035
weighted avg       0.96      0.96      0.96      1035


 confussion matrix:
 [[703  17]
 [ 26 289]]


In [74]:
#vote of count method for output ensemble
from scipy import stats
submit = [submit_log[i]+submit_rf[i]+submit_xgm[i]+submit_svm[i]+submit_ada[i] for i in range(len(submit_svm))]
submit = np.asarray(submit)

submit[np.where(submit==1)] = 0
submit[np.where(submit==2)] = 0
submit[np.where(submit==3)] = submit_log[np.where(submit==3)]
submit[submit > 3] = 1




In [43]:
# ### Save as .csv

# In[ ]:

df = pd.DataFrame(submit)
df.index += 1
df['Id'] = df.index
df.columns = ['Category', 'Id']
df.to_csv("submit_new.csv",header=True,columns=['Id','Category'],index = False)